In [22]:
import pandas as pd

#Tratando os dados
url = "DAT_ASCII_EURJPY_M1_2018.csv"
df = pd.read_csv(url, delimiter = ';', 
                 header = None, 
                 names = ["Data", "Open", "High", "Low", "Close", "Volume"], 
                 parse_dates = ["Data"])

df.set_index("Data", inplace = True) 
df.index.names = [None]
df = df.drop("Volume", axis=1)                                                    
df.index = df.index.tz_localize('EST').tz_convert('Brazil/West').tz_localize(None)

#Transformando 1min em 5min
df_ohlc = df.resample('5Min').agg({'Open': 'first', 
                                 'High': 'max', 
                                 'Low': 'min', 
                                 'Close': 'last'})

#Calculando media móvel exponencial(100 períodos) e fator de inclinação (positivo/negativo)

df_ohlc['EMA100'] = df_ohlc['Close'].ewm(span = 100, adjust = False, min_periods = 100).mean()

df_ohlc.loc[(df_ohlc.EMA100 > df_ohlc.EMA100.shift(1)), 'AC'] = 1
df_ohlc.loc[(df_ohlc.EMA100 < df_ohlc.EMA100.shift(1)), 'AC'] = -1
df_ohlc.loc[(df_ohlc.EMA100 == df_ohlc.EMA100.shift(1)), 'AC'] = 0

#Calculando Banda de Bollinger (20 períodos (p), 2.5 std(std))
p = 20
std = 2.5
bbsma = f'SMA{p}'
bbs = f'BS{p}'
bbi = f'BI{p}'
df_ohlc[bbsma] = df_ohlc['Close'].rolling(p).mean()
df_ohlc[bbs] = df_ohlc[bbsma] + (std * df_ohlc['Close'].rolling(p).std())
df_ohlc[bbi] = df_ohlc[bbsma] - (std * df_ohlc['Close'].rolling(p).std())

#Condicionando o setup de entrada

df_ohlc['Result'] = 0

#TruePositive
df_ohlc.loc[((df_ohlc.AC.shift(1) == 1) 
             & (df_ohlc.Low < df_ohlc[bbi]) 
             & (df_ohlc.Close > df_ohlc[bbi]) 
             & (df_ohlc.Open > df_ohlc[bbi])), 'Result'] = 1

#TrueNegative
df_ohlc.loc[((df_ohlc.AC.shift(1) == -1) 
             & (df_ohlc.High > df_ohlc[bbs]) 
             & (df_ohlc.Close < df_ohlc[bbs]) 
             & (df_ohlc.Open < df_ohlc[bbs])), 'Result'] = 1

#FalsePositive
df_ohlc.loc[((df_ohlc.AC.shift(1) == 1) 
             & (df_ohlc.Low < df_ohlc[bbi]) 
             & (df_ohlc.Close <= df_ohlc[bbi]) 
             & (df_ohlc.Open > df_ohlc[bbi])), 'Result'] = -1

#FalseNegative
df_ohlc.loc[((df_ohlc.AC.shift(1) == -1) 
             & (df_ohlc.High > df_ohlc[bbs]) 
             & (df_ohlc.Close >= df_ohlc[bbs]) 
             & (df_ohlc.Open < df_ohlc[bbs])), 'Result'] = -1

#Calculando Winrate

df_ohlc = df_ohlc[100:]

WinCount = len(df_ohlc.loc[(df_ohlc["Result"] == 1)])
LossCount = len(df_ohlc.loc[(df_ohlc["Result"] == -1)])
Winrate = (WinCount / (WinCount + LossCount)) * 100

print('WinCount: ', WinCount)
print('LossCount: ', LossCount)
print('Winrate: ', Winrate)

#Gráfico com indicações do indicador


WinCount:  1112
LossCount:  505
Winrate:  68.76932591218305


In [ ]:
# #Plotando gráfico de candlesticks (https://youtu.be/19yyasfGLhk?t=318)
# def plot_p(df):
#     import matplotlib.pyplot as plt
#     from mpl_finance import candlestick2_ohlc
#     fig, ax = plt.subplots(figsize=(20, 10))
#     candlestick2_ohlc(ax,
#                       df_ohlc['Open'].values,
#                       df_ohlc['High'].values,
#                       df_ohlc['Low'].values,
#                       df_ohlc['Close'].values,
#                       width = 0.6,
#                       colorup = 'g',
#                       colordown = 'r',
#                       alpha = 1, )
#     plt.show()
    
# plot_p(df_ohlc[:300])

In [ ]:
import matplotlib.pyplot as plt
num = 2000
plt.figure(figsize = (15,10))
plt.grid(True)
plt.plot(df_ohlc['Close'][:num], label = 'Close')
plt.plot(df_ohlc[bbsma][:num], label = bbsma)
plt.plot(df_ohlc['EMA100'][:num], label = 'EMA100')
plt.legend(loc = 2)

In [ ]:
import matplotlib.pyplot as plt
num = 500
plt.figure(figsize = (15,10))
plt.grid(True)
plt.plot(df_ohlc['Close'][:num], label = 'Close')
plt.plot(df_ohlc[bbsma][:num], label = bbsma)
plt.plot(df_ohlc[bbs][:num], label = bbs)
plt.plot(df_ohlc[bbi][:num], label = bbi)
plt.legend(loc = 2)